## Custom written code to generate files for Figure 7 Liu 2023

Run under the analysis2 environment

Preparation for Figure 7. Innervation centers in MO regions


In [33]:
import os

import pandas as pd

import numpy as np

#import Neuron_analysis as na

import skimage
from skimage import io
from skimage import filters
from skimage import segmentation
from skimage import measure

import re
from tqdm import tqdm

import nrrd

In [31]:

def find_mousename(text):
    #finds name of mouse that follows the typical LSENS pattern: two letters followed by 3 numbers, ie AL000
    a= re.search('[a-zA-Z]{2}[0-9]{2,3}', text)
    return a[0]

def isolate_region_2(regions_list):
    
    '''useful for isolating one layer of multiple region, ie MOs 2/3 and MOp 2/3''' 
    
    #child_ids= atlas_labels.loc[atlas_labels['structure_id_path'].str.contains('|'.join(regions_list), case=False)].id
    # child_ids

    # find all regions with regionX inside the structure id path
    # for example 315 - isocortex, 985- primary motor region
    
    this_mask= np.zeros(annot_h.shape)
    # create empty array to store the mask
    for i in tqdm(regions_list):
        this_mask[annot_h==i]=1
        #loop through all the region id and assign value of 1
    # empty_array now is a matrix of 1 and 0s
    
    return this_mask

def find_contour_fractionmax2d(axon_array,fraction):
    
        
    ''' input axon stack as 2d numpy array (id, already in horizontal sum projection) and fraction of maximal intensity wish to compute (ie. 0.95)
    returns binary image and contour on horizontal sumprojection'''

    
    test1= skimage.filters.gaussian(axon_array, sigma=4)
    # apply gaussian filter

    #io.imshow(test1) 
    #io.show()
    # show sum projection in horizontal view
    
    contour= np.max(test1) * fraction
    threshold= test1>contour
    # find X% of max intensity and threshold

    #io.imshow(threshold) 
    #io.show()
    
    outline= skimage.segmentation.find_boundaries(threshold, mode='inner').astype(np.uint8)
    #find outline of the segmentation

    #io.imshow(outline) 
    #io.show()
    
    return threshold, outline

def find_fractionmax2d(axon_array, fraction):
    
    ''' input axon stack as 2d numpy array (id, already in horizontal sum projection) and fraction of maximal intensity wish to compute (ie. 0.95)
    returns binary image and contour on horizontal sumprojection'''

    
    test1= skimage.filters.gaussian(axon_array, sigma=4)
    # apply gaussian filter

    #io.imshow(test1) 
    #io.show()
    # show sum projection in horizontal view
    
    contour= np.max(test1) * fraction
    threshold= test1>contour
    # find X% of max intensity and threshold

    #io.imshow(threshold1) 
    #io.show()
    
    return threshold

def conv_stereo(ML, AP, realum):
    '''input data points in terms of pixels and real pixel size in micrometers 
    calculates stereotaxic coordinate and return as ML, AP values'''
    bregma= [227.5, 215]
    # bregma coordiantes in horizontal view, where image dimension is [456, 528]
        
    tmpmedlat= abs(bregma[0]-ML)
    tmpantpos=bregma[1]- AP
    
    factor= realum/1000
    medlat=tmpmedlat*factor
    antpos= tmpantpos*factor
    # convert pixels to real micrometers
    
    return round(medlat,2), round(antpos,2)

def find_outline(binary_image):
    outline= skimage.segmentation.find_boundaries(binary_image, mode='inner').astype(np.uint8)
    #find outline of the segmentation

    #io.imshow(outline) 
    #io.show()
    
    return outline


In [ ]:
working_directory= r'E:\CodeTest' 
# this directory should contain folders Horizontal_Axon, ARA_25_micron_mhd_ccf2017, and file injection_sites_results_expanded.xlsx

In [5]:
file_name = f'{working_directory}\ARA_25_micron_mhd_ccf2017\\annotation_25.nrrd'
data_array, metadata = nrrd.read(file_name)
# read atlas files

annot_h=np.moveaxis(data_array, 1, 0)
print('Converted to horizontal atlas with shape', annot_h.shape)
# reslice corontal atlas to horizontal atlas that is consistent with our sample's orientation

atlas_labels=pd.read_csv('{working_directory}\ARA_25_micron_mhd_ccf2017\labels.csv')
#read labels

Converted to horizontal atlas with shape (320, 528, 456)


In [6]:
# include axons from both L2/3 and L5

region_id= [943,962, 648, 767]
#atlas id for MOp2/3, MOs2/3, MOp5, MOs5

# cannot be string to use isolate region2
region_mask= isolate_region_2(region_id)
# generate mask outside of the loop since it will be the same!

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.42it/s]


In [41]:
site='s2'
# change to 's1' or 's2' and repeat

indir= f'{working_directory}\\Horizontal_Axon\\{site}\\npy'
outdir= f'{working_directory}\ML_centroids\\{site}'
os.makedirs(outdir)

files= os.listdir(indir)

In [42]:
# start working with the 95% max segmentations
# we get lots of measurements using this segmentation: 95% contours, centroid locations, sum axon # within the 95% max (in the horizontal plane)


MO_centroid_ML= []
MO_centroid_AP= []

values_95=[]
sample_list=[]

fractionmax=0.95

for i in tqdm(files): 
    
    file_path=os.path.join(indir,i)
    this_sample= find_mousename(i)
    # find the sample name
    
    sample_list.append(this_sample)
    # append sample name to sample_list
    
    axons=np.load(file_path)
    axons[0,0,0]=0
    axons[0,1,0]=0
    # load npy file and revert these two indices back to 0
    
    axons_masked= axons*region_mask
    # multiply 3D axon stack with 3D mask stack
    
    axons_h= axons_masked.sum(0)
    # create horizontal sum projection from the masked stack
    skimage.io.imsave(f'{outdir}\\{this_sample}_sum_projection.tiff', axons_h,check_contrast=False)
    # save this horizontal sum projection
    
    
    thresh= find_fractionmax2d(axons_h, fractionmax)
    # get segmentation and outline of the horizontal sum projection
    
    label_img = skimage.measure.label(thresh)
    # create connected component labels
    
    if label_img.max()==0:
        # when there is no axons at all and thus no 95% contour 
        MO_centroid_ML.append(np.nan)
        MO_centroid_AP.append(np.nan)
        
        values_95.append(0)
        
        # just assign nan as centroid and 0 as sum axons inside the 95% contour
        
    else: 
        # when there is axons
        
        largestCC = label_img == np.argmax(np.bincount(label_img.flat)[1:])+1
        # return only the largest connected component, this is is a boolean array that can be used as mask (https://stackoverflow.com/questions/47540926/get-the-largest-connected-component-of-segmentation-image)
        # for some samples, there are two connected components satisfying the 95% percent intensity 

        axons_95= largestCC*axons_h
        values_95.append(axons_95.sum())
        # get total number of axons within this segmentation, append to list

        CC_img = largestCC*1
        #convert the boolean into 0 and 1 for skimage.measure.regionprops to work
        region=skimage.measure.regionprops(CC_img)
        # measure connected components of thres in order to get centroid location
        
        outline=find_outline(CC_img)
        outline=outline.astype(np.int8)
        # convert to 8 bit, for easier merge in image J since the centoid image next will be 8 bit
        skimage.io.imsave(f'{outdir}\\{this_sample}_contour_{fractionmax}.tiff', outline,check_contrast=False)
        # save the contour image

        temp= np.zeros_like(axons_h)
        # intialize empty array to plot the centroid

        
        for k in region:
            # there should be only one region

            AP_pix,ML_pix= k.centroid
            # extract coordinates for centroid

            AP_pix=int(AP_pix)
            ML_pix=int(ML_pix)

            temp[AP_pix,ML_pix]=1
            # plot the centroid for each connected component

            ML,AP= conv_stereo(ML_pix,AP_pix,25)
            # convert pixel values to seterotaxis coordinate
            #print(f'MO axon centroid for {this_sample} is ML= {ML}mm,AP= {AP}mm')

            MO_centroid_ML.append(ML)
            MO_centroid_AP.append(AP)
            # append ML and AP position to list, for later making excel file

    temp=temp.astype(np.int8)
    skimage.io.imsave(f'{outdir}\\{this_sample}_centroid_{fractionmax}.tiff', temp,check_contrast=False)
    # save the centroid image


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [02:16<00:00,  7.19s/it]


In [43]:
# get image and areas of 75% contour

area_75=[]
sample_list2=[]

fractionmax=0.75

for i in tqdm(files): 
    
    file_path=os.path.join(indir,i)
    this_sample= find_mousename(i)
    # find the sample name
    
    axons=np.load(file_path)
    axons[0,0,0]=0
    axons[0,1,0]=0
    # load npy file and revert these two indices back to 0
    
    axons_masked= axons*region_mask
    
    axons_h= axons_masked.sum(0)
    # create horizontal sum projection from the masked stack
    
    thresh,outline= find_contour_fractionmax2d(axons_h, fractionmax)
    
    outline=outline.astype(np.int8)
    # convert to 8 bit, for easier merge in image J since the centoid image next will be 8 bit
    skimage.io.imsave(f'{outdir}\\{this_sample}_contour_{fractionmax}.tiff', outline,check_contrast=False)
    # save the contour image
    
    label_img = skimage.measure.label(thresh)
    region=skimage.measure.regionprops(label_img)
    
    
    area_list=[]
    for k in region:
        # loop over connected components (in most cases there is just 1), but for multiple connected components with 75% contour, sum the areas together

        this_area=k.area
        area_list.append(this_area)

    sample_list2.append(this_sample)
    area_75.append(sum(area_list))
    # sum the areas together
    
    # append 75% contour area to list


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:57<00:00,  3.04s/it]


In [44]:
d = {'sample': sample_list, 'ML': MO_centroid_ML,'AP': MO_centroid_AP, 'area_75': area_75, 'axons_95': values_95}
df = pd.DataFrame(data=d)
# create data frame with the info


df.to_csv(f'{outdir}_{site}.csv')

df

,sample,ML,AP,area_75,axons_95
0,AL274,0.89,-0.40,159,0.360329
1,AL278,0.74,1.10,673,110.528395
2,AL280,0.74,0.98,837,50.393258
3,AL281,1.01,1.48,85,27.415730
4,AL286,0.89,1.50,41,1.648148
5,AL288,1.01,1.50,238,40.455639
6,AL290,0.96,1.65,79,4.093976
7,AL292,0.74,1.40,95,2.317536
8,AL303,0.71,1.33,240,8.232679
9,AL310,0.99,1.90,116,0.488576
